In [ ]:
import numpy as np

class Particle(object):
    """
    A charged particle in an electric field.
    """
    def __init__(self, q, E):
        self.q = q
        self.E = E

    def __repr__(self):
        return 'Particle(%f, %f)' % (self.q, self.E)

    @property
    def force(self):
        return np.dot(self.E, self.q)


# Add comments on the API

In [ ]:
from numba import types

class ParticleType(types.Type):
    def __init__(self):
        super(ParticleType, self).__init__(name='Particle')

particle_type = ParticleType()


In [ ]:
from numba.extending import typeof_impl

@typeof_impl.register(Particle)
def typeof_index(val, c):
    return particle_type

In [ ]:
from numba.extending import type_callable

@type_callable(Particle)
def type_particle(context):
    def typer(q, E):
        if isinstance(q, types.Float) and isinstance(E, types.Float):
            return particle_type
    return typer

In [ ]:
from numba.extending import models, register_model

@register_model(ParticleType)
class ParticleModel(models.StructModel):
    def __init__(self, dmm, fe_type):
        members = [
            ('q', types.float64),
            ('E', types.float64),
            ]
        models.StructModel.__init__(self, dmm, fe_type, members)

In [ ]:
from numba.extending import make_attribute_wrapper

make_attribute_wrapper(ParticleType, 'q', 'q')
make_attribute_wrapper(ParticleType, 'E', 'E')

In [ ]:
from numba.extending import overload_attribute

@overload_attribute(ParticleType, "force")
def get_force(particle):
    def getter(particle):
        return particle.E * particle.q
    return getter

In [ ]:
from numba.extending import lower_builtin
from numba.core import cgutils

@lower_builtin(Particle, types.Float, types.Float)
def impl_particle(context, builder, sig, args):
    typ = sig.return_type
    q, E = args
    particle = cgutils.create_struct_proxy(typ)(context, builder)
    particle.q = q
    particle.E = E
    return particle._getvalue()

In [ ]:
from numba.extending import unbox, NativeValue

@unbox(ParticleType)
def unbox_particle(typ, obj, c):
    """
    Convert a Particle object to a native particle structure.
    """
    q_obj = c.pyapi.object_getattr_string(obj, "q")
    E_obj = c.pyapi.object_getattr_string(obj, "E")
    particle = cgutils.create_struct_proxy(typ)(c.context, c.builder)
    particle.q = c.pyapi.float_as_double(q_obj)
    particle.E = c.pyapi.float_as_double(E_obj)
    c.pyapi.decref(q_obj)
    c.pyapi.decref(E_obj)
    is_error = cgutils.is_not_null(c.builder, c.pyapi.err_occurred())
    return NativeValue(particle._getvalue(), is_error=is_error)

In [ ]:
from numba.extending import box

@box(ParticleType)
def box_particle(typ, val, c):
    """
    Convert a native particle structure to an Particle object.
    """
    particle = cgutils.create_struct_proxy(typ)(c.context, c.builder, value=val)
    q_obj = c.pyapi.float_from_double(particle.q)
    E_obj = c.pyapi.float_from_double(particle.E)
    class_obj = c.pyapi.unserialize(c.pyapi.serialize_object(Particle))
    res = c.pyapi.call_function_objargs(class_obj, (q_obj, E_obj))
    c.pyapi.decref(q_obj)
    c.pyapi.decref(E_obj)
    c.pyapi.decref(class_obj)
    return res

In [ ]:
from numba import jit

# @jit(nopython=True)
# def inside_particle(particle, x):
#     return particle.q <= x < particle.E

@jit(nopython=True)
def particle_force(particle):
    return particle.force

@jit(nopython=True)
def sum_particles(i, j):
    return Particle(i.q + j.q, i.E + j.E)

In [ ]:
import numpy as np
print(sum_particles(Particle(1, 2), Particle(2, 6)))

In [ ]:
print(particle_force(Particle(5, 10)))